In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [17]:
#超参
num_epochs = 10
batch_size = 100
learning_rate = 0.001

In [18]:
#MNIST 数据集
train_dataset = dsets.MNIST(root='../data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root = '../data/', train=False, transform=transforms.ToTensor(), download=True)

In [19]:
#导入数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [20]:
#构建CNN模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
cnn = CNN()
cnn.cuda()

CNN (
  (layer1): Sequential (
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (layer2): Sequential (
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (fc): Linear (1568 -> 10)
)

In [21]:
#定义损失函数和优化函数
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

In [22]:
#训练网络
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        #前向+后向+优化
        optimizer.zero_grad()
        outputs = cnn(images)
        
        #调用损失函数计算输出结果与真实值的差
        loss = criterion(outputs, labels)
        
        #后向传播
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/10], Iter [100/600] Loss: 0.1484
Epoch [1/10], Iter [200/600] Loss: 0.0482
Epoch [1/10], Iter [300/600] Loss: 0.0911
Epoch [1/10], Iter [400/600] Loss: 0.1019
Epoch [1/10], Iter [500/600] Loss: 0.0621
Epoch [1/10], Iter [600/600] Loss: 0.0462
Epoch [2/10], Iter [100/600] Loss: 0.0362
Epoch [2/10], Iter [200/600] Loss: 0.1191
Epoch [2/10], Iter [300/600] Loss: 0.0346
Epoch [2/10], Iter [400/600] Loss: 0.0371
Epoch [2/10], Iter [500/600] Loss: 0.0195
Epoch [2/10], Iter [600/600] Loss: 0.0444
Epoch [3/10], Iter [100/600] Loss: 0.0386
Epoch [3/10], Iter [200/600] Loss: 0.0516
Epoch [3/10], Iter [300/600] Loss: 0.0078
Epoch [3/10], Iter [400/600] Loss: 0.0279
Epoch [3/10], Iter [500/600] Loss: 0.0305
Epoch [3/10], Iter [600/600] Loss: 0.0220
Epoch [4/10], Iter [100/600] Loss: 0.0048
Epoch [4/10], Iter [200/600] Loss: 0.0148
Epoch [4/10], Iter [300/600] Loss: 0.0067
Epoch [4/10], Iter [400/600] Loss: 0.0128
Epoch [4/10], Iter [500/600] Loss: 0.0152
Epoch [4/10], Iter [600/600] Loss:

In [23]:
#网络测试
cnn.eval()
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images).cuda()
    outputs = cnn(images)
    _, predictd = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predictd == labels.cuda()).sum()
    
print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Test Accuracy of the model on the 10000 test images: 99 %


In [24]:
#保存模型
torch.save(cnn.state_dict(), 'cnn.pkl')